In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
from missing_clf import *

In [3]:
house_value_train = 'input/house_pricing/train.csv'
df = pd.read_csv(house_value_train) 
print(df.shape)

(1460, 81)


In [4]:
missing_values = df.isnull().sum()
cols_with_missing = missing_values[missing_values > 0]
print(cols_with_missing)

LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [5]:
to_drop_rows = cols_with_missing[cols_with_missing <= 100].index
to_drop_rows

Index(['MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Electrical', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond'],
      dtype='object')

In [6]:
to_drop_rows = list(cols_with_missing[cols_with_missing <= 100].index)
df = df.dropna(subset=to_drop_rows, axis=0)
to_drop_cols = list(cols_with_missing[cols_with_missing > 100].index)
df = df.drop(to_drop_cols, axis=1)
df.reset_index(drop=True, inplace=True)
df.shape

(1338, 74)

In [7]:
numerical_cols = df.select_dtypes(include=['number']).columns
df = df[numerical_cols]
df.shape

(1338, 37)

In [8]:
df = df.drop('Id', axis=1)

In [9]:
df.describe()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,...,1338.000000,1338.000000,1338.000000,1338.000000,1338.00000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,56.136024,10706.294469,6.219731,5.596413,1973.029148,1985.668909,110.360239,464.234679,49.218236,582.494768,...,99.384903,47.782511,21.263827,3.585949,16.43423,3.010463,42.932735,6.331839,2007.805680,186761.782511
std,41.252576,10336.621126,1.324472,1.078124,29.563540,20.296463,185.604816,458.792420,166.196584,439.950528,...,127.537065,65.362562,60.843964,30.224622,58.05159,41.961337,508.056255,2.699437,1.330691,78913.847668
min,20.000000,1300.000000,2.000000,2.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,2006.000000,35311.000000
25%,20.000000,7744.000000,5.000000,5.000000,1956.000000,1968.000000,0.000000,0.000000,0.000000,248.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,5.000000,2007.000000,135000.000000
50%,50.000000,9600.000000,6.000000,5.000000,1976.000000,1994.500000,0.000000,413.000000,0.000000,489.000000,...,6.000000,28.000000,0.000000,0.000000,0.00000,0.000000,0.000000,6.000000,2008.000000,168500.000000
75%,70.000000,11760.750000,7.000000,6.000000,2001.000000,2004.000000,174.000000,733.000000,0.000000,815.750000,...,174.500000,70.000000,0.000000,0.000000,0.00000,0.000000,0.000000,8.000000,2009.000000,220000.000000
max,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,...,857.000000,547.000000,552.000000,508.000000,480.00000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
from missingness import MissingnessParams

from missingness import apply_missingness, MissingnessParamsGenerator
from missing_clf import classify_missing_values
from tqdm import tqdm

results = []

it = MissingnessParamsGenerator(
        features=df.drop('SalePrice', axis=1).columns, initial_seed=42, n_experiments=10000
)
for params in tqdm(it):
    try:
        modified_df = apply_missingness(df, params)
        acc = classify_missing_values(modified_df, params.target_feature, seed=params.random_state)
        results.append({**params, 'acc': acc})
    except:
        continue

results = pd.DataFrame(results)
results.to_csv("house_price.csv", index=False)
results


0it [00:00, ?it/s]

360it [00:15, 17.37it/s]

In [ ]:
from train import train_rf
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline

import numpy as np
import pandas as pd
from tqdm import tqdm
from missingness import MissingnessParams, apply_missingness

params = MissingnessParams(
    mechanism="MAR",
    strategy="basic",
    random_state=42,
    target_feature="total_sulfur_dioxide",
    missing_rate=0.5,
    condition_feature="residual_sugar",
)

estimators = {
    'bayes': BayesianRidge(),
    'rf': RandomForestRegressor(
        n_estimators=4,
        max_depth=10,
        bootstrap=True,
        max_samples=0.5,
        n_jobs=2,
        random_state=0,
    ),
    'kernel': make_pipeline(
        Nystroem(kernel="polynomial", degree=2, random_state=0),
        Ridge(alpha=1e3)
    ),
    'knn': KNeighborsRegressor(n_neighbors=15)
}

imputators = {
    'drop_rows': FunctionTransformer(lambda x: x.dropna(axis=0)),
    'drop_col': FunctionTransformer(lambda x: x.dropna(axis=1)),
    'mean': SimpleImputer(missing_values=np.nan, strategy='mean'),
    'median': SimpleImputer(missing_values=np.nan, strategy='median'),
    'zero': SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0),
    'knn_uniform': KNNImputer(n_neighbors=5, weights="uniform"),
    'knn_distance': KNNImputer(n_neighbors=5, weights="distance"),
    ** {f"iterative_{estimator_name}": 
        IterativeImputer(random_state=42, estimator=impute_estimator, max_iter=25, tol=1e-3) 
            for estimator_name, impute_estimator in estimators.items()}
}

results = []

modified_df = apply_missingness(df, params)

for impute_name, imputer in tqdm(imputators.items()):
    imputed_df = pd.DataFrame(imputer.fit_transform(modified_df), columns=modified_df.columns)
    X, y = imputed_df.drop(columns=["quality"]), imputed_df["quality"] - 3
    model, acc = train_rf(X, y)
    results.append({'acc': acc, 'impute': impute_name})

X, y = df.drop(columns=["quality"]), df["quality"] - 3
model, acc = train_rf(X, y)
results.append({'acc': acc, 'impute': 'baseline'})

results = pd.DataFrame(results)
results